In [70]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import sqlalchemy
import sqlite3
import os

import warnings
warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")

In [71]:
os.getcwd()
new_path = '/Users/sarasharifzadeh/anaconda3/APROJECTS/projectfiles'
os.chdir(new_path)

In [72]:
conn = sqlite3.connect('basic_examples.db')

### StructuredQueryLanguage

A declarative language (not imperative) in which you "say **what** you want, **not how** to get it.

* **Declarative:** I want a table with columns "X" & "Y" constructed from tables "A" & "B" where the values in "Y" are greater than 100
* **Imperative:** For each record in table "a" find the corresponding record in table "B" then drop the records where "Y" is less than or equal to 100, then return the "X" & "Y" values

<details>
    <summary>[<b>Relational Terminology</b>]</summary>
    
* Database: set of relations (one or more tableS)
* Relation (Table):
    * Schema: descriptions of columns, their types, and constraints
    * instance: data satisfying the schema
* Attribute (Column)
* Tuple (Record, Row)
* Schema of database: set of schemas of its relations  

</details>

<details>
<summary><h2><b>DOCUMENT OUTLINE</b></h2></summary>
    
1. Full Overview / Reference 
2. Creating & Populating Tables
3. Querying into Tables
    1. basic overview
    2. examples 
5. Keys & Joins
6. smthn
    1. Basic Examples: Dish Table
    2. Complex Example: db
</details>


### BASIC SQL Query Syntax


```
SELECT <column list>
FROM <table>
[WHERE <predicate>]
    [AND <predicate>]
[GROUP BY <column list>]
[HAVING <predicate>]
[ORDER BY <column list>]
[LIMIT <number of rows>]
[OFFSET <number of rows>]
```

In [98]:
# to-do:
# include create
# examples for having, order by, offset
# deep review for groupby, join 

## Creating & Populating Tables
### **`SELECT`**
Specifying what column(s) we would like to extract from a given table, or how to create them.

* **`AS`** to rename as an alias
```sql
        SELECT engineSpeed AS speed
```    
* **`DISTINCT`** selects only unique values in a column, causing duplicate entries to be removed
```sql
        SELECT DISTINCT nameId 
``` 
* **`CAST(col AS dtype)`** casting col to a dtype: int, float, str
```sql
        SELECT genres, CAST(startYear AS int) AS year
```
* Common Numeric Functions: **`COUNT()`, `AVG()`, `SUM()`, `LENGTH()`,`MAX()`/ `MIN()`, `AVG()`, `POW()`, `LEAST()`, `MOD()`**,  to be used with groupby aggregation (or single condition)
  
```sql
        SELECT genres, SUM((averageRating)*numVotes)/SUM(numVotes) AS avgGenreRating
```

* **`CASE`** or **`IF(condition, value_if_true, value_if_false)`** to select upon a condition... (if else)

```sql
 CASE WHEN <cond> THEN <fill val>
      WHEN <other cond> THEN <fill val>
         ...
      ELSE <yet another val>
    END AS <col name> or END
```

**Example**: grade report with columns (name, grade, mark) but dont want names of students receiving a grade less than 8.

```sql
SELECT CASE WHEN grade < 8 THEN NULL              v.         SELECT IF(grade < 8, NULL, name), grade, marks
          ELSE name
       END,
       grade, marks
...
```
**Example**: If a movie was filmed before 1950, it is "old". Otherwise, if a movie was filmed before 2000, it is "mid-aged". Else, a movie is "new". 
```sql
SELECT titleType, startYear,
CASE WHEN startYear < 1950 THEN "old"
     WHEN startYear < 2000 THEN "mid-aged"
     ELSE "new"
     END AS movie_age
FROM Title
LIMIT 5
```

|titleType	|startYear	|movie_age|
|:-----|:----:|----:|
|short|	1902|	old|
|movie|	1915|	old|
|movie|	1978|	mid-aged|
|movie|	2024|	new|

## Filtering Output

### Filtering Rows
### **`WHERE`**
selecting only some rows of a table, filtering rows on a condition. 


* Conditionals: **`AND`,`OR`, `NOT`**
    * Comparison Operators: =, <, <, >=, <=, <>
    * Arithmetic: +, -, *, /, %
```sql
SELECT time, title, genre FROM movies
WHERE time > 2000 AND genre NOT = "Drama"
```

* **`IN`, `IS`, `IS NOT`, `NULL`**
```sql
SELECT name FROM city                                              SELECT name FROM city 
WHERE name IN ("New York", "Los Angeles", "San Francisco")         WHERE time IS NOT NULL 
...
```

* **`REGEXP`**
```sql
SELECT DISTINCT city From station 
WHERE city REGEXP '[aeiou]$' 
    AND city REGEXP '^[aeiou]';
```

* **`LIKE`, `EXISTS`**

```sql
SELECT titleType, primaryTitle                                     SELECT primaryTitle
FROM Title                                                         FROM Title
WHERE primaryTitle LIKE "%Star Wars%"                v.            WHERE primaryTitle LIKE "Star Wars" 
...
```

```sql
SELECT titleType, primaryTitle                                     SELECT primaryTitle
FROM Title                                                         FROM Title
WHERE primaryTitle LIKE "%Star Wars%"                v.            WHERE primaryTitle LIKE "Star Wars" 
...
```

**`%`** is a wildcard operator, it looks for any character any number of times. Without it, only exact matches return
|titleType  |primaryTitle |
|:-----|----:|
|movie |Star Wars: Episode IV - A New Hope |
|movie |Star Wars: Episode V - The Empire Strikes Back |


```sql
SELECT name FROM city
WHERE time IS NOT NULL 
...
```

* Subqueries:

```sql
SELECT <cols>
FROM <table>
    [WHERE <predicate>]  {
       [SELECT <col> FROM <table>]
       [WHERE <predicate>]
          [AND]
    }
...
```

### Filtering Groups
**The difference between WHERE & HAVING is:** filtering rows v. filtering groups. 

### **`HAVING`**
Filters groups by applying some condition across all rows in each group. We interpret it as a way to keep only the groups HAVING some condition.  

* `WHERE` filters rows, precedes having
* `HAVING` filters groups

**Ouptut comparison below:** In the first, the code groups the dishes by type, and only keeps those groups where the max cost is less than 8. In the second, the code filters for rows where the cost is less than 8, then does the grouping. This results in `WHERE` returning more rows than `HAVING` in this case because it includes as additional entree.


```sql
SELECT type, COUNT(*)                                               SELECT type, COUNT(*)
FROM Dish                                                           FROM Dish
GROUP BY type                            v.                         WHERE cost < 8 
HAVING MAX(cost) < 8;                                               GROUP BY type;
```


### Sorting Output
### **`ORDER BY`**
**`ORDER BY`**: default ASC, change to `DESC`
* Ordering by two columns at once will sort the table by the first listed column & use values in 2nd to break the tie.

**Example**: order by character length first, then break ties by city name, DESC included in both! 
```sql
SELECT city, LENGTH(city)
FROM station
ORDER BY LENGTH(city) DESC, city DESC
LIMIT 1;
```

**`LIMIT`**: restrict output to the specified number of rows

**`OFFSET`**: indicates the index at which LIMIT should start at. 

#### **RANDOM USEFUL QUERIES I'VE SEEN**

1. **Example**: given **a range** of grades, **assign** a students mark to the grade

```sql
SELECT name, grade, mark
FROM students JOIN grades
WHERE mark BETWEEN min_mark AND max_mark
```
2. **Example**: Query the list of CITY names **ending** then **starting with vowels** (i.e., a, e, i, o, or u) from STATION. unique.

```sql
SELECT DISTINCT city From station
WHERE city REGEXP '[aeiou]$';
```

```sql
SELECT DISTINCT city FROM station
 WHERE city REGEXP '^[aeiou]';
```

3. **Example**: all cities where id **numbers are even**
   
```sql
SELECT DISTINCT city FROM station 
WHERE id%2=0  (or)  WHERE  MOD(id,2) = 0
```

4. **Example**: **order students by the last 3 characters** of their name, if two or more have same last 3 ending, break tie with id.
```sql
SELECT name FROM students
ORDER BY RIGHT(name, 3), id; 
```


5. **Example**: query the names of all the continents & respective **average city populations rounded down to the nearest integer**.

```sql
SELECT country.continent, FLOOR(AVG(city.population))
FROM city 
INNER JOIN country ON city.countrycode = country.code
GROUP BY country.continent;
```


# BELOW IS UNCLEANED UNORGANIZED

In [86]:
#load the module
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


### Very Basic SQL learning: A small table of dishes

Using this data from Berkeley's Data100 course.

In [87]:
%sql sqlite:///basic_examples.db

In [88]:
%%sql 
SELECT * FROM sqlite_master WHERE type='table';

 * sqlite:///basic_examples.db
   sqlite:///imdbmini.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,7,"CREATE TABLE sqlite_sequence(name,seq)"
table,Dragon,Dragon,2,"CREATE TABLE Dragon ( name TEXT PRIMARY KEY, year INTEGER CHECK (year >= 2000), cute INTEGER)"
table,Dish,Dish,4,"CREATE TABLE Dish ( name TEXT PRIMARY KEY, type TEXT, cost INTEGER CHECK (cost >= 0))"
table,Scene,Scene,6,"CREATE TABLE Scene ( id INTEGER PRIMARY KEY AUTOINCREMENT, biome TEXT NOT NULL, city TEXT NOT NULL, visitors INTEGER CHECK (visitors >= 0), created_at DATETIME DEFAULT (DATETIME('now')))"


##### Previewing the Dish Table

In [89]:
%%sql
SELECT * FROM Dish;

 * sqlite:///basic_examples.db
   sqlite:///imdbmini.db
Done.


name,type,cost
ravioli,entree,10
ramen,entree,13
taco,entree,7
edamame,appetizer,4
fries,appetizer,4
potsticker,appetizer,4
ice cream,dessert,5


In [90]:
%%sql
SELECT DISTINCT(type) AS mealType FROM Dish;

 * sqlite:///basic_examples.db
   sqlite:///imdbmini.db
Done.


mealType
entree
appetizer
dessert


In [91]:
%%sql
SELECT name, type, MAX(cost) AS expensiveMeal
FROM Dish;

 * sqlite:///basic_examples.db
   sqlite:///imdbmini.db
Done.


name,type,expensiveMeal
ramen,entree,13


**`WHERE`**
selecting only some rows of a talbe, filtered on a condition. 

**The difference between WHERE & HAVING is:** filtering rows v. filtering groups. "WHERE" is used to filter rows.

* Conditionals: `AND`,`OR`, `NOT`, `IN`, `IS`, `IS NOT`, `NULL`, `LIKE`, `EXISTS`
* Comparison Operators: =, <, <, >=, <=, <>
* Arithmetic: +, -, *, /, %
* Subquery:

```
SELECT <cols>
FROM <table>
    [WHERE <predicate>] {
    [SELECT <col> FROM <table>]
    [WHERE <predicate>]
        [AND]}
```

In [60]:
%%sql
SELECT *
FROM Dish
WHERE type IN ("appetizer");

   sqlite:///basic_examples.db
 * sqlite:///imdbmini.db
(sqlite3.OperationalError) no such table: Dish
[SQL: SELECT *
FROM Dish
WHERE type IN ("appetizer");]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [61]:
%%sql 
SELECT * FROM Dish
WHERE type = 'entree' 
AND cost <= 10;

   sqlite:///basic_examples.db
 * sqlite:///imdbmini.db
(sqlite3.OperationalError) no such table: Dish
[SQL: SELECT * FROM Dish
WHERE type = 'entree' 
AND cost <= 10;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [92]:
%%sql
SELECT * FROM Dish
LIMIT 2
OFFSET 1;

 * sqlite:///basic_examples.db
   sqlite:///imdbmini.db
Done.


name,type,cost
ramen,entree,13
taco,entree,7


In [93]:
%%sql
SELECT * FROM Dish
ORDER BY cost, type

 * sqlite:///basic_examples.db
   sqlite:///imdbmini.db
Done.


name,type,cost
edamame,appetizer,4
fries,appetizer,4
potsticker,appetizer,4
ice cream,dessert,5
taco,entree,7
ravioli,entree,10
ramen,entree,13


In [95]:
%%sql
SELECT type, COUNT(*)
FROM Dish
GROUP BY type
HAVING MAX(cost) < 8;

 * sqlite:///basic_examples.db
   sqlite:///imdbmini.db
Done.


type,COUNT(*)
appetizer,3
dessert,1


In [103]:
%%sql
SELECT type, COUNT(*)
FROM Dish
WHERE cost < 8
GROUP BY type;

 * sqlite:///basic_examples.db
   sqlite:///imdbmini.db
Done.


type,COUNT(*)
appetizer,3
dessert,1
entree,1


#### Matching Text 

In [104]:
%sql sqlite:///imdbmini.db

In [105]:
%%sql
SELECT titleType, primaryTitle
FROM Title
WHERE primaryTitle LIKE "%Star Wars%"
LIMIT 10;

   sqlite:///basic_examples.db
 * sqlite:///imdbmini.db
Done.


titleType,primaryTitle
movie,Star Wars: Episode IV - A New Hope
movie,Star Wars: Episode V - The Empire Strikes Back
movie,Star Wars: Episode VI - Return of the Jedi
movie,Star Wars: Episode I - The Phantom Menace
movie,Star Wars: Episode II - Attack of the Clones
movie,Star Wars: Episode III - Revenge of the Sith
tvSeries,Star Wars: Clone Wars
tvSeries,Star Wars: The Clone Wars
movie,Star Wars: The Clone Wars
movie,Star Wars: Episode VII - The Force Awakens


#### New Columns with Conditional Statements

In [106]:
%%sql
/* If a movie was filmed before 1950, it is "old"
Otherwise, if a movie was filmed before 2000, it is "mid-aged"
Else, a movie is "new" */

SELECT titleType, startYear,
CASE WHEN startYear < 1950 THEN "old"
     WHEN startYear < 2000 THEN "mid-aged"
     ELSE "new"
     END AS movie_age
FROM Title
LIMIT 5;

   sqlite:///basic_examples.db
 * sqlite:///imdbmini.db
Done.


titleType,startYear,movie_age
short,1902,old
movie,1915,old
movie,1920,old
movie,1921,old
movie,1922,old
